### Source: https://www.youtube.com/playlist?list=PLJ-sIDAvcXl7VaU-AR27Kwiw52Q9UTDEn

In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD, RDFS, RDF, OWL, XMLNS
import pathlib
from dotenv import load_dotenv
from settings import path_base

In [2]:
# Own ontology:
# path_to_ontology = pathlib.Path(path_base, "models/Ontologies/Ontology3.ttl").as_posix()
path_to_ontology = pathlib.Path(path_base, "models/Ontologies/Ontology4.ttl").as_posix()
# From tutorial:
# path_to_ontology = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl"

In [3]:
# Custom Namespace
NS = Namespace('http://onto.neo4j.com/rail#')

In [4]:
def getLocalPart(uri):
  pos = -1
  pos = uri.rfind('#')
  if pos < 0 :
    pos = uri.rfind('/')
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[pos+1:]

def getNamespacePart(uri):
  pos = -1
  pos = uri.rfind('#')
  if pos < 0 :
    pos = uri.rfind('/')
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[0:pos+1]

In [31]:
# Load the ontology from the Turtle file
g = Graph()
g.parse(path_to_ontology, format='ttl')

key_properties = {'Company': 'LEI'}

# Get DatatypeProperties for each Class
data_type_properties = dict()
for (subj1, pred1, obj1) in g.triples((None, RDF.type, OWL.Class)):
    for (subj2, pred2, obj2) in g.triples((None, RDFS.domain, subj1)):
        for (subj3, pred3, obj3) in g.triples((subj2, RDF.type, OWL.DatatypeProperty)):
            if not subj1 in data_type_properties:
               data_type_properties[subj1] = list()
            data_type_properties[subj1].append(subj3)

print(data_type_properties)

{rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Company'): [rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#LEI'), rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#companyName'), rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#reportingPeriod')], rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Emission'): [rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#BaselineValue'), rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#BaselineYear')]}


In [39]:
# Write Cypher-Queries for DatatypeProperties
nodes = list()
# export_ns = set()
# export_mappings = {}

for uri_class, uri_property_list in data_type_properties.items():
    #########
    # export_ns.add(getNamespacePart(row.curi))
    # export_mappings[getLocalPart(row.curi)] = str(row.curi)
    node = getLocalPart(uri_class)
    s1 = f"""
    UNWIND $json_data AS data
    MERGE (n:{node})
    """
    props = list()
    for prop in uri_property_list:
        p = getLocalPart(prop)
        if node in key_properties and p == key_properties[node]:
            constr = f"CREATE CONSTRAINT ON (n:{node}) REQUIRE n.{p} IS UNIQUE"
            props.append(constr)
        props.append(f"SET n.{p} = data.{p}")
    props.append("RETURN count(*) as total")
    s2 = ' \n\t'.join(props)
    nodes.append(s1+s2)

for c in nodes:
    print(c)
    print('----')


    UNWIND $json_data AS data
    MERGE (n:Company)
    CREATE CONSTRAINT ON (n:Company) REQUIRE n.LEI IS UNIQUE 
	SET n.LEI = data.LEI 
	SET n.companyName = data.companyName 
	SET n.reportingPeriod = data.reportingPeriod 
	RETURN count(*) as total
----

    UNWIND $json_data AS data
    MERGE (n:Emission)
    SET n.BaselineValue = data.BaselineValue 
	SET n.BaselineYear = data.BaselineYear 
	RETURN count(*) as total
----


In [7]:
# Write Cypher-Queries for Relationships between Classes
# Get DatatypeProperties for each Class
relationships = list()
for (subj1, pred1, obj1) in g.triples((None, RDF.type, OWL.ObjectProperty)):
    for (subj2, pred2, obj2) in g.triples((subj1, RDFS.domain, None)):
        for (subj3, pred3, obj3) in g.triples((subj1, RDFS.range, None)):
            relationships.append({'SOURCE': obj2, 'TARGET': obj3, 'REL': subj1})

print(relationships)

[{'SOURCE': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Asset'), 'TARGET': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Energy'), 'REL': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#consumes')}, {'SOURCE': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#FromNuclearSources'), 'TARGET': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#NuclearWaste'), 'REL': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#disposes')}, {'SOURCE': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Asset'), 'TARGET': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#Scope1'), 'REL': rdflib.term.URIRef('http://www.semanticweb.org/rainer/ontologies/2023/10/Ontology4#emits')}, {'SOURCE': rdflib.term.URIRef('http://www.semanticweb.or

In [11]:
rels = list()
for rel in relationships:
    rs = f"""
    unwind $records AS record 
    match (source:{getLocalPart(rel['SOURCE'])})
    match (target:{getLocalPart(rel['TARGET'])})
    merge (source)-[r:{getLocalPart(rel['REL'])}]->(target)
    return count(*) as total
    """
    rels.append(rs)

for i in rels:
    print(i)


    unwind $records AS record 
    match (source:Asset)
    match (target:Energy)
    merge (source)-[r:consumes]->(target)
    return count(*) as total
    

    unwind $records AS record 
    match (source:FromNuclearSources)
    match (target:NuclearWaste)
    merge (source)-[r:disposes]->(target)
    return count(*) as total
    

    unwind $records AS record 
    match (source:Asset)
    match (target:Scope1)
    merge (source)-[r:emits]->(target)
    return count(*) as total
    

    unwind $records AS record 
    match (source:Company)
    match (target:EnvironmentalFootprint)
    merge (source)-[r:impacts]->(target)
    return count(*) as total
    

    unwind $records AS record 
    match (source:FromFossileSources)
    match (target:Scope2)
    merge (source)-[r:indirectlyEmits]->(target)
    return count(*) as total
    

    unwind $records AS record 
    match (source:Company)
    match (target:FinancialReport)
    merge (source)-[r:isIn]->(target)
    return count(*) 

SyntaxError: f-string: expecting '}' (865439066.py, line 3)